In [1]:
## testing printing output from console
import subprocess

cmd = [ 'echo', '"Welcome to my PySpark analysis of the StackOverflow dataset"' ]
output = subprocess.Popen( cmd, stdout=subprocess.PIPE ).communicate()[0]
print(output)

b'"Welcome to my PySpark analysis of the StackOverflow dataset"\n'


# Load Libraries

In [2]:
import gc #garbage collection
import time
import numpy as np
import pandas as pd
# set pd float display options to not have trailing 0's
from datetime import datetime

# Load PySpark

In [3]:
%run -i '1-load-pyspark.py'

The Spark UI, version 2.4.3, is available at: http://192.168.0.26:4040/ and the defaultParallelism is 4


# Load easyFunctions and Transformers

In [4]:
## easy functions
%run -i 'load_parquet_data.py'
%run -i 'convert_stackov_csv_to_pyarrow_parquet.py'
%run -i 'export_parquet_data.py'
%run -i 'count_total_questions.py'
%run -i 'show_save_results.py'
%run -i 'show_spark_df.py'
%run -i 'show_date_range.py'
%run -i 'trim_betw_dates.py'

## pipeline transformers
%run -i 'nltkWordPunctTokeniser.py'
%run -i 'nltkSenteniser.py'

# Choose Datasets

In [5]:
## these are the forum labels that will appear in graphs/tables etc.
data_array = [
    #'Mar-09',
    #'Apr-09',
    
    'May-09',
    'Jun-09',
    'Jul-09',
    'Aug-09',
    'Sep-09',
    
    #'Oct-09', #breaks here
    #'Nov-09',
    #'Dec-09'
]

# Download XML Data

In [6]:
## call download script - EDIT to feed in user data_array
#!bash 0-dataset-download.sh

# Convert to Parquet

In [7]:
%%time
print(datetime.now().time())

## convert stackoverflow .json files to .parquet
#for i in data_array:
#    s = i.lower()
#    convert_stackov_csv_to_pyarrow_parquet(strDirect=f'initial-data/{s}.stackexchange.com/')

15:48:04.327083
CPU times: user 623 µs, sys: 636 µs, total: 1.26 ms
Wall time: 1.07 ms


In [8]:
## loop through data_array to convert to other fora to .parquet
#for i in data_array:
#    s = i.lower()
#    !python convert-xml-to-parquet.py "$s"

# Load Initial or Clean Data

In [9]:
%%time
print(datetime.now().time())
datasets = load_parquet_data(dataArray=data_array, kind='clean', printSchema=False) #, gcpath='gs://bucket-brad-project/')

15:48:04.346875
CPU times: user 5.07 ms, sys: 2.71 ms, total: 7.79 ms
Wall time: 4.23 s


In [10]:
## count questions before trimming
count_total_questions(dataArray=data_array, datasetDict=datasets)

May-09: 26026
Jun-09: 28555
Jul-09: 32752
Aug-09: 32998
Sep-09: 33268

Total: 153599


In [11]:
#########################################################################
### Create shifted score variable
#########################################################################

# import lit function
from pyspark.sql.functions import lit, expr

# find minimum of score for shifting
minp = 1000
for i in data_array:
    if (min(datasets[i].select('score').rdd.flatMap(lambda x: x).collect()) < minp):
        # absolute value because min score is negative
        minp = min( datasets[i].select('score').rdd.flatMap(lambda x: x).collect() )

# adjust minimum to incremented absolute value
minp = abs(minp) + 2

# create new shifted score variable
for i in data_array:
    datasets[i] = datasets[i].withColumn('new_column', lit(minp)).\
    withColumn('score_shift', expr("new_column + score"))

#########################################################################
### Create logged score and viewcount variables
#########################################################################

# import log function
from pyspark.sql.functions import log

# create new logged score variable
for i in data_array:
    datasets[i] = datasets[i].withColumn('score_shift_log', log('score_shift'))
    
# create new logged viewcount variable
for i in data_array:
    datasets[i] = datasets[i].withColumn('viewcount_log', log('viewcount'))

# Clean and Trim Data

In [12]:
%%time
print(datetime.now().time())
#%run -i '2-clean-datasets.py'

15:48:16.283165
CPU times: user 322 µs, sys: 76 µs, total: 398 µs
Wall time: 360 µs


In [13]:
## show range of dates for datasets before trimming
show_date_range(dataArray=data_array, datasetDict=datasets)

May-09:
2009-05-01 00:02:42.960000
2009-05-31 23:56:47.303000

Jun-09:
2009-06-01 00:06:57.250000
2009-06-30 23:59:14.543000

Jul-09:
2009-07-01 00:01:35.740000
2009-07-31 23:58:46.123000

Aug-09:
2009-08-01 00:02:10.593000
2009-08-31 23:59:12.223000

Sep-09:
2009-09-01 00:01:17.477000
2009-09-30 23:59:36.613000



In [14]:
## trim data between uniform date range
#datasets = trim_betw_dates(dataArray=data_array, datasetDict=datasets, dates=('2014-06-01', '2017-01-01'))

In [15]:
## show range of dates for datasets after trimming
show_date_range(dataArray=data_array, datasetDict=datasets)

May-09:
2009-05-01 00:02:42.960000
2009-05-31 23:56:47.303000

Jun-09:
2009-06-01 00:06:57.250000
2009-06-30 23:59:14.543000

Jul-09:
2009-07-01 00:01:35.740000
2009-07-31 23:58:46.123000

Aug-09:
2009-08-01 00:02:10.593000
2009-08-31 23:59:12.223000

Sep-09:
2009-09-01 00:01:17.477000
2009-09-30 23:59:36.613000



In [16]:
## count questions after trimming
count_total_questions(dataArray=data_array, datasetDict=datasets)

May-09: 26026
Jun-09: 28555
Jul-09: 32752
Aug-09: 32998
Sep-09: 33268

Total: 153599


# Counts and LDA Feature Engineering

In [17]:
%%time
# about 16 min with 10 iterations and params for 29000 questions
# FAILS for large datasets
print(datetime.now().time())
#%run -i '3-feat-engineering.py'

###
# TRY different values of alpha and rho
# TRY different values of K
# TRY 'em' optimizer instead of 'online'

15:48:21.513020
CPU times: user 865 µs, sys: 552 µs, total: 1.42 ms
Wall time: 1.45 ms


# Export Clean Data

In [18]:
%%time
# about 12 min
print(datetime.now().time())
#export_parquet_data(dataArray=data_array, datasetDict=datasets, folder='cleaner-data')

15:32:19.049283
CPU times: user 4.15 ms, sys: 2.89 ms, total: 7.04 ms
Wall time: 9.61 ms


# EDA

In [ ]:
%%time
print(datetime.now().time())
%run -i '4-final-eda.py'

# Train/Test Splits

In [18]:
## garbage collector to speed up computation
collected = gc.collect()
print(f'Garbage collector: collected {collected} objects.')

Garbage collector: collected 304 objects.


In [19]:
%%time
print(datetime.now().time())
%run -i '6a-rand-train-test-split-50.py'
#%run -i '6b-time-train-test-split-50.py'

## check standard deviations of variables
print('\nStandard deviations are:\n')
for i in data_array:
    tr = round( pd.to_numeric(train[i].describe('score_shift_log').select('score_shift_log').toPandas().iloc[2][0]), 3 )
    te = round( pd.to_numeric(test[i].describe('score_shift_log').select('score_shift_log').toPandas().iloc[2][0]), 3 )
    perc = round( (te/tr - 1)*100, 3 )
    s = i.title() + ' & ' + str(tr) + ' & ' + str(te) + ' & ' + str(perc) + ' \\\\'
    print(s)

15:48:21.633158

Standard deviations are:

May-09 & 0.427 & 0.425 & -0.468 \\
Jun-09 & 0.434 & 0.427 & -1.613 \\
Jul-09 & 0.403 & 0.4 & -0.744 \\
Aug-09 & 0.362 & 0.388 & 7.182 \\
Sep-09 & 0.35 & 0.365 & 4.286 \\
CPU times: user 80.1 ms, sys: 22.8 ms, total: 103 ms
Wall time: 6.52 s


# Imports

In [36]:
## pyspark sql functions
from pyspark.sql.functions import lit, avg, col

## for evaluation metrics
from pyspark.ml.evaluation import RegressionEvaluator

## for pipelines
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF, StandardScaler, VectorAssembler, VectorSlicer

## for model
from pyspark.ml.regression import LinearRegression

## for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Choose Target, Modeling Setups

In [21]:
## choose target variable
target = 'score_shift_log'

## setup linear regression model on just viewcount
lr = LinearRegression(maxIter=100,
                      #regParam=0.3,
                      #elasticNetParam=0.8,
                      featuresCol='features',
                      labelCol=target)
                      #predictionCol='preds')
    
## choose evaluation metric
ev_metric = 'rmse' #'mae'

## choose number of CV folds
num_folds = 2

## set up grid for parameter tuning: 
'''NEEDED, BUT SLOWS DOWN'''
# Ravi et al use L2, aka ridge, aka elasticNetParam=0
# regParam is the value of lambda
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [1e-2, 1.]) \
    .addGrid(lr.regParam, [1e-2, 1.]) \
    .build()

## blank tuning grid
noGrid = ParamGridBuilder() \
    .build()

## prediction column names
mean_preds = 'mean_preds'
viewc_preds = 'viewc_preds'
leng_preds = 'leng_preds'
token_preds = 'token_preds'
topic_preds = 'topic_preds'
final_preds = 'final_preds'

'''
## features not used
datet_variables = ['clean_date']

## assemblers not used
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')'''

"\n## features not used\ndatet_variables = ['clean_date']\n\n## assemblers not used\ndatet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')"

# Silly Mean Model

In [30]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

In [31]:
## create mean dictionaries
tr_means = {}

## calculate the mean of each forum, using ONLY training set
for i in data_array:
    tr_means[i] = train[i].select(target).rdd.flatMap(lambda x: x).mean() #.agg(avg(col(target))).collect()[0][0]

## create dictionaries for training and testing (baseline) rmse 
base = {}
tr_rmse = {}

## setup rmse evaluator
model_evaluator = RegressionEvaluator(metricName=ev_metric, labelCol=target, predictionCol=mean_preds)

## modelling
for i in data_array:

    ## initial variable for timing
    t0 = time.time()
    
    ## train silly mean model by assigning training set mean for training and testing predictions
    train[i] = train[i].withColumn(mean_preds, lit(tr_means[i]))
    test[i] = test[i].withColumn(mean_preds, lit(tr_means[i]))

    ## evaluate silly mean model, on both training and testing set
    tr_rmse[i] = round( model_evaluator.evaluate(train[i]), 2 )
    base[i] = round( model_evaluator.evaluate(test[i]), 2 )

    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m mean\033[0m model is {base[i]}")

    ## record time taken
    timet = round( time.time() - t0, 0 )
    
    ## store as dictionary inside RESULTS dictionary, initiating dataset name entries first
    RESULTS[i.title()]['0silly_mean.0tr_rmse'] = tr_rmse[i]
    RESULTS[i.title()]['0silly_mean.1rmse'] = base[i]
    RESULTS[i.title()]['0silly_mean.2timet'] = timet
    
## record results
show_save_results(RESULTS)

The root-mean-square error of May-09's mean model is 0.42
The root-mean-square error of Jun-09's mean model is 0.43
The root-mean-square error of Jul-09's mean model is 0.4
The root-mean-square error of Aug-09's mean model is 0.39
The root-mean-square error of Sep-09's mean model is 0.36


,0silly_mean.0tr_rmse,0silly_mean.1rmse,0silly_mean.2timet
May-09,0.43,0.42,1.0
Jun-09,0.43,0.43,1.0
Jul-09,0.40,0.40,1.0
Aug-09,0.36,0.39,1.0
Sep-09,0.35,0.36,1.0


\begin{tabular}{lrrr}
\toprule
{} &  0silly\_mean.0tr\_rmse &  0silly\_mean.1rmse &  0silly\_mean.2timet \\
\midrule
May-09 &                  0.43 &               0.42 &                 1.0 \\
Jun-09 &                  0.43 &               0.43 &                 1.0 \\
Jul-09 &                  0.40 &               0.40 &                 1.0 \\
Aug-09 &                  0.36 &               0.39 &                 1.0 \\
Sep-09 &                  0.35 &               0.36 &                 1.0 \\
\bottomrule
\end{tabular}



# Viewcount Model

In [32]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

In [33]:
%%time
print(datetime.now().time())

########################
##### CHOOSE FEATS #####
########################

## define features to predict on
numic_variables = ['viewcount_log']

## numerical columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])


####################
##### MODELING #####
####################

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## linear regression model
lr = LinearRegression(maxIter=100,
                      #regParam=1,
                      #elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol=viewc_preds)

## setup rmse evaluator
model_evaluator = RegressionEvaluator(metricName=ev_metric, labelCol=target, predictionCol=viewc_preds)

## set up cross validation for parameter tuning
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=noGrid,
                          evaluator=model_evaluator,
                          numFolds=num_folds)
## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    
    ## fitting on train and predicting on train/test
    tr_rmse = round( model_evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( model_evaluator.evaluate(cvmodel.transform(test[i])), 2 )
        
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m viewcount\033[0m model is {rmse}")

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 0 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['1viewcount.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['1viewcount.1rmse'] = rmse
    RESULTS[i.title()]['1viewcount.2imprv'] = impr
    RESULTS[i.title()]['1viewcount.3timet'] = timet
    
## record results
show_save_results(RESULTS)

15:52:19.047921
The root-mean-square error of May-09's viewcount model is 0.31
The root-mean-square error of Jun-09's viewcount model is 0.32
The root-mean-square error of Jul-09's viewcount model is 0.3
The root-mean-square error of Aug-09's viewcount model is 0.29
The root-mean-square error of Sep-09's viewcount model is 0.28


,1viewcount.0tr_rmse,1viewcount.1rmse,1viewcount.2imprv,1viewcount.3timet
May-09,0.31,0.31,26.19,7.0
Jun-09,0.32,0.32,25.58,6.0
Jul-09,0.30,0.30,25.00,6.0
Aug-09,0.28,0.29,25.64,6.0
Sep-09,0.27,0.28,22.22,6.0


\begin{tabular}{lrrrr}
\toprule
{} &  1viewcount.0tr\_rmse &  1viewcount.1rmse &  1viewcount.2imprv &  1viewcount.3timet \\
\midrule
May-09 &                 0.31 &              0.31 &              26.19 &                7.0 \\
Jun-09 &                 0.32 &              0.32 &              25.58 &                6.0 \\
Jul-09 &                 0.30 &              0.30 &              25.00 &                6.0 \\
Aug-09 &                 0.28 &              0.29 &              25.64 &                6.0 \\
Sep-09 &                 0.27 &              0.28 &              22.22 &                6.0 \\
\bottomrule
\end{tabular}

CPU times: user 1.03 s, sys: 268 ms, total: 1.29 s
Wall time: 32.1 s


## Count Model

In [ ]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

In [37]:
%%time
print(datetime.now().time())

########################
##### CHOOSE FEATS #####
########################

## define features to predict on
numic_variables = ['body_word_cnt', 'titl_word_cnt', 'body_char_cnt', 
                   'titl_char_cnt', 'body_sent_cnt', 'titl_sent_cnt']

## NUMERICAL columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

## create PROCESSING pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
lr = LinearRegression(maxIter=100,
                      #regParam=1,
                      #elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol=leng_preds)

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName=ev_metric, labelCol=target, predictionCol=leng_preds)

## set up cross validation for parameter tuning
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=num_folds)

## create models dict for recording gridsearch params
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m counts\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 0 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2counts.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['2counts.1rmse'] = rmse
    RESULTS[i.title()]['2counts.2imprv'] = impr
    RESULTS[i.title()]['2counts.3regular'] = reg_param
    RESULTS[i.title()]['2counts.4elastic'] = ela_param    
    RESULTS[i.title()]['2counts.5timet'] = timet
    
## record results
show_save_results(RESULTS)

15:59:05.538687
The root-mean-square error of May-09's counts model is 0.42
The root-mean-square error of Jun-09's counts model is 0.43
The root-mean-square error of Jul-09's counts model is 0.4
The root-mean-square error of Aug-09's counts model is 0.39
The root-mean-square error of Sep-09's counts model is 0.36


,1viewcount.0tr_rmse,1viewcount.1rmse,1viewcount.2imprv,1viewcount.3timet,2counts.0tr_rmse,2counts.1rmse,2counts.2imprv,2counts.3regular,2counts.4elastic,2counts.5timet
May-09,0.31,0.31,26.19,7.0,0.43,0.42,-0.0,0.0,0.0,11.0
Jun-09,0.32,0.32,25.58,6.0,0.43,0.43,-0.0,0.0,0.0,12.0
Jul-09,0.30,0.30,25.00,6.0,0.40,0.40,-0.0,0.0,0.0,12.0
Aug-09,0.28,0.29,25.64,6.0,0.36,0.39,-0.0,0.0,0.0,13.0
Sep-09,0.27,0.28,22.22,6.0,0.35,0.36,-0.0,0.0,0.0,12.0


\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  1viewcount.0tr\_rmse &  1viewcount.1rmse &  1viewcount.2imprv &  1viewcount.3timet &  2counts.0tr\_rmse &  2counts.1rmse &  2counts.2imprv &  2counts.3regular &  2counts.4elastic &  2counts.5timet \\
\midrule
May-09 &                 0.31 &              0.31 &              26.19 &                7.0 &              0.43 &           0.42 &            -0.0 &               0.0 &               0.0 &            11.0 \\
Jun-09 &                 0.32 &              0.32 &              25.58 &                6.0 &              0.43 &           0.43 &            -0.0 &               0.0 &               0.0 &            12.0 \\
Jul-09 &                 0.30 &              0.30 &              25.00 &                6.0 &              0.40 &           0.40 &            -0.0 &               0.0 &               0.0 &            12.0 \\
Aug-09 &                 0.28 &              0.29 &              25.64 &                6.0 &              0.36 &          

## Text Model

In [38]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

In [ ]:
%%time
print(datetime.now().time())

########################
##### CHOOSE FEATS #####
########################

## define features to predict on
textt_variables = ['title', 'clean_body']

## textual columns
# tokenising text cols with custom transformer
nltk_tokeniser_body = nltkWordPunctTokeniser(
    inputCol='clean_body', outputCol='body_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

nltk_tokeniser_title = nltkWordPunctTokeniser(
    inputCol='title', outputCol='titl_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

# count occurence of tokens, i.e. create dfm
cnt_vectrizr_body = CountVectorizer(inputCol='body_words', outputCol='body_raw_feats') #!!! minDF???
cnt_vectrizr_title = CountVectorizer(inputCol='titl_words', outputCol='titl_raw_feats')

# create IDF dfm
idf_body = IDF(inputCol="body_raw_feats", outputCol="body_feats")
idf_title = IDF(inputCol="titl_raw_feats", outputCol="titl_feats")

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['body_feats', 'titl_feats'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[  #inputCols=['datet_data']
    nltk_tokeniser_body, 
    nltk_tokeniser_title,
    cnt_vectrizr_body,
    cnt_vectrizr_title,
    idf_body,
    idf_title,
    process_assembler
])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
lr = LinearRegression(maxIter=100,
                      #regParam=1,
                      #elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol=token_preds)

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName=ev_metric, labelCol=target, predictionCol=token_preds)

## set up cross validation for parameter tuning
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=num_folds)

## create models dict for recording gridsearch params
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m tokens\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 0 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2tokens.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['2tokens.1rmse'] = rmse
    RESULTS[i.title()]['2tokens.2imprv'] = impr
    RESULTS[i.title()]['2tokens.3regular'] = reg_param
    RESULTS[i.title()]['2tokens.4elastic'] = ela_param
    RESULTS[i.title()]['2tokens.5timet'] = timet
    
## record results
show_save_results(RESULTS)

16:00:05.792556


In [ ]:
## check predictions aren't constant
models[data_array[0]].transform(test[data_array[0]]).select('token_preds').take(10)

# LDA Features Model

In [ ]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

In [ ]:
%%time
print(datetime.now().time())

########################
##### CHOOSE FEATS #####
########################

## define features to predict on
numic_variables = ['btd[0]', 'btd[1]', 'btd[2]', 'btd[3]', 'btd[4]', 
                   'btd[5]', 'btd[6]', 'btd[7]', 'btd[8]', 'btd[9]', 
                   'std[0]', 'std[1]', 'std[2]', 'std[3]', 'std[4]', 
                   'std[5]', 'std[6]', 'std[7]', 'std[8]', 'std[9]',
                   'ttd[0]', 'ttd[1]', 'ttd[2]', 'ttd[3]', 'ttd[4]', 
                   'ttd[5]', 'ttd[6]', 'ttd[7]', 'ttd[8]', 'ttd[9]'] 

## numerical columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
lr = LinearRegression(maxIter=100,
                      #regParam=1,
                      #elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol=topic_preds)

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName=ev_metric, labelCol=target, predictionCol=topic_preds)

## set up cross validation for parameter tuning
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=num_folds)

## create models dict for recording gridsearch params
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m topics\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 0 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2topics.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['2topics.1rmse'] = rmse
    RESULTS[i.title()]['2topics.2imprv'] = impr
    RESULTS[i.title()]['2topics.3regular'] = reg_param
    RESULTS[i.title()]['2topics.4elastic'] = ela_param
    RESULTS[i.title()]['2topics.5timet'] = timet
    
## record results
show_save_results(RESULTS)

# Final Model

In [ ]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

In [ ]:
%%time
print(datetime.now().time())

########################
##### CHOOSE FEATS #####
########################

## define features to predict on
numic_variables = ['btd[0]', 'btd[1]', 'btd[2]', 'btd[3]', 'btd[4]', 
                   'btd[5]', 'btd[6]', 'btd[7]', 'btd[8]', 'btd[9]', 
                   'std[0]', 'std[1]', 'std[2]', 'std[3]', 'std[4]', 
                   'std[5]', 'std[6]', 'std[7]', 'std[8]', 'std[9]',
                   'ttd[0]', 'ttd[1]', 'ttd[2]', 'ttd[3]', 'ttd[4]', 
                   'ttd[5]', 'ttd[6]', 'ttd[7]', 'ttd[8]', 'ttd[9]',
                   'body_word_cnt', 'titl_word_cnt', 'body_char_cnt', 
                   'titl_char_cnt', 'body_sent_cnt', 'titl_sent_cnt']

### NUMERICAL columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

## create PROCESSING pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[  #inputCols=['datet_data']
    numic_pipeline,
    process_assembler
])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
lr = LinearRegression(maxIter=100,
                      #regParam=1,
                      #elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol=final_preds)

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName=ev_metric, labelCol=target, predictionCol=final_preds)

## set up cross validation for parameter tuning
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=num_folds)

## create models dict for recording gridsearch params
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m final\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 0 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['3final.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['3final.1rmse'] = rmse
    RESULTS[i.title()]['3final.2imprv'] = impr
    RESULTS[i.title()]['3final.3regular'] = reg_param
    RESULTS[i.title()]['3final.4elastic'] = ela_param
    RESULTS[i.title()]['3final.5timet'] = timet
    
## record results
show_save_results(RESULTS)

In [ ]:
## check predictions aren't constant
models[data_array[0]].transform(test[data_array[0]]).select('final_preds').take(10)

In [ ]:
fin

# Save predictions

In [ ]:
(trained_pipeline
 .transform(datasets['english'])
 .select(
    indep_text_variables + ["prediction"]
 )
 .write
 .parquet("linreg_prediction.parquet")
)

In [ ]:
linreg_predictions = spark.read.parquet("linreg_prediction.parquet")

In [ ]:
linreg_predictions.toPandas().head()

In [ ]:
linreg_predictions.select("prediction").describe().toPandas()

# Save pipelines

In [ ]:
from joblib import dump, load
dump(estimator_pipeline, 'pipeline.joblib') 

reloaded = load("pipeline.joblib")

#Now we can predict directly!

reloaded.predict(X)[:10]

In [ ]:
## save models DOESN'T WORK BECAUSE: 'NLTKWordPunctTokenizer' object has no attribute '_to_java'
for i in data_array:
    param_dict[i].save(f'{i}-pipeline') 

# Convert notebook to python file

In [ ]:
!jupyter nbconvert --to script 0-master-notebook-pipelines.ipynb